In [1]:
# Load the required libraries and functions

%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import random
from numpy.random import randn
from math import exp
from openpyxl import Workbook
import matplotlib.pyplot as plt
import copy
from datetime import datetime
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# Below is the Create Model Function

def CreateModel(Day, pi):
    
    # Load the Data
    Distance = pd.read_excel('Gates.xlsx', index_col=0)
    Arrival = pd.read_excel('Arrivals.xlsx', sheetname = (Day - 1))
    Departure = pd.read_excel('Departures.xlsx', sheetname = (Day - 1))
    
    
    Arrival_shape = Arrival.shape[0]
    Departure_shape = Departure.shape[0]
    ind = np.arange(Departure_shape) + Arrival_shape
    Departure = Departure.set_index(ind)
    

    # Create Transfer Passengers Data
    P = DataFrame(0, index= np.arange(Arrival_shape), columns = (ind))
    np.random.seed(100)
    for i in np.arange(Arrival_shape):
        initial = Arrival['Passengers'][i]
        for j in np.arange(Departure_shape):
            if (Arrival['Arrival Time'][i] + 40 < Departure['Departure Time'].iloc[j] and 
                    Departure['Departure Time'].iloc[j] - Arrival['Arrival Time'][i] <= 480):
                x = Arrival['Passengers'][i]
                y = np.random.randint(0,15)
                if x - y >= (1 - pi) * initial:
                    P.iloc[[i],[j]] = y
                    Arrival['Passengers'][i] = x - y                       


    frames = [Arrival, Departure]
    Flights = pd.concat(frames)
    Flights_sort = Flights.sort_values(['Departure Time'], ascending=[True])
    
    # Define I, J and nVar
    J = Distance.shape[0] - 1
    I = Arrival_shape + Departure_shape
    nVar = I + J - 1
    
    
    # Output
    return {'Distance': Distance, 'Arrival': Arrival, 'Day': Day, 'Departure': Departure, 'Arrival_shape': Arrival_shape, 
            'Departure_shape': Departure_shape, 'ind': ind, 'pi': pi, 'P': P, 'Flights': Flights, 'Flights_sort': Flights_sort,
            'J': J, 'I': I, 'nVar': nVar}

In [3]:
# Below is the Create Initial Solution Function

def CreateInitialSol(Model):

    J = Model['Distance'].shape[0] - 1
    I = Model['Arrival_shape'] + Model['Departure_shape']
    nVar = I + J - 1   
    
    # Create Intial Solution using a Greedy Approach
    Del = list(np.arange(I,nVar))
    diff = J
    Gate1 =[]
    for i in np.arange(I/J):
        Gate1.append(Model['Flights_sort'].iloc[i*diff].name)
    Gate1
    Gate2 =[]
    for i in np.arange(I/J):
        Gate2.append(Model['Flights_sort'].iloc[i*diff + 1].name)
    Gate2
    Gate3 =[]
    for i in np.arange(I/J):
        Gate3.append(Model['Flights_sort'].iloc[i*diff + 2].name)
    Gate3
    Gate4 =[]
    for i in np.arange(I/J):
        Gate4.append(Model['Flights_sort'].iloc[i*diff + 3].name)
    Gate4
    Gate5 =[]
    for i in np.arange(I/J):
        Gate5.append(Model['Flights_sort'].iloc[i*diff + 4].name)
    Gate5
    Gate6 =[]
    for i in np.arange(I/J):
        Gate6.append(Model['Flights_sort'].iloc[i*diff + 5].name)
    Gate6
    Gate7 =[]
    for i in np.arange(I/J):
        Gate7.append(Model['Flights_sort'].iloc[i*diff + 6].name)
    Gate7
    Gate8 =[]
    for i in np.arange(I/J):
        Gate8.append(Model['Flights_sort'].iloc[i*diff + 7].name)
    Gate8
    Gate9 =[]
    for i in np.arange(I/J):
        Gate9.append(Model['Flights_sort'].iloc[i*diff + 8].name)
    Gate9
    Gate10 =[]
    for i in np.arange(I/J):
        Gate10.append(Model['Flights_sort'].iloc[i*diff + 9].name)
    Gate10
    Gate11 =[]
    for i in np.arange(I/J):
        Gate11.append(Model['Flights_sort'].iloc[i*diff + 10].name)
    Gate11
    Gate12 =[]
    for i in np.arange(I/J):
        Gate12.append(Model['Flights_sort'].iloc[i*diff + 11].name)
    Gate12
    Gate13 =[]
    for i in np.arange(I/J):
        Gate13.append(Model['Flights_sort'].iloc[i*diff + 12].name)
    Gate13
    Gate14 =[]
    for i in np.arange(I/J):
        Gate14.append(Model['Flights_sort'].iloc[i*diff + 13].name)
    Gate14
    Gates = [list(Gate1), list(Gate2), list(Gate3), list(Gate4), list(Gate5), list(Gate6), list(Gate7), list(Gate8), 
                 list(Gate9), list(Gate10), list(Gate11), list(Gate12), list(Gate13), list(Gate14)]
    rem = I - diff*(I/J)
    for i in np.arange(rem):
            Gates[i].append(Model['Flights_sort'].iloc[J * (I/J) + i].name)


    # Create a intial 'q' using above 'Gates'
    q = []
    for i in np.arange(len(Gates) - 1):
        qi = Gates[i] + [Del[i]]
        q = q + qi
    qi = Gates[i + 1]
    q = q + qi
    
    
    # Create Xij Dataframe 
    X = DataFrame(0, index= np.arange(I), columns = np.arange(1,J + 1))
    for i in np.arange(1,J + 1):
        for j in (Gates[i - 1]):
            X[i][j] = 1
    
    # Output
    return {'Del': Del, 'diff': diff, 'Gates': Gates, 'rem': rem, 'q': q, 'X': X}

In [4]:
# Below is the funcition for checking the feasibility of inital solution

def CheckInitialSol(Model, qi):
    
    Gates = qi['Gates']
    Flights = Model['Flights']
    q = qi['q']
    
    gate_emp = []
    for i in Gates:
        L = len(i)
        gate_emp.append(L) 

    clash = 0
    apron = []
    for g in Gates:    
        for i in g:
            ai = Flights['Arrival Time'][i]
            di = Flights['Departure Time'][i]
            for j in g:
                if i == j:
                    continue
                else:
                    aj = Flights['Arrival Time'][j]
                    dj = Flights['Departure Time'][j]
                    if (dj - ai) * (di - aj) <= 0:
                        continue
                    else:
                        clash = clash + 1
    clash = clash/2
    if clash <= 0: 
        temp = "Pass"
    else:
        temp = "Fail"
        
    empty = min(gate_emp)
        
    # Output    
    return {'Gates': Gates, 'clash': clash, 'temp': temp, 'empty': empty, 'q': q}

In [5]:
# Below is the fucntion to calculate the cost of the inital solution

def IntialCost(Model, qi):
    
    Flights = Model['Flights']
    Distance = Model['Distance']
    X = qi['X']
    Arrival_shape = Model['Arrival_shape']
    P = Model['P']
    ind = Model['ind']
    Gates = qi['Gates']
    pi = Model['pi']
    
    
    # Calulate the Intial Cost of arriving and departing passengers
    CostTransfer = 0
    CostOrigin = 0
    for i in np.arange(Flights.shape[0]):
        xo = Flights.ix[i]
        Passo = xo['Passengers']
        gato = X.ix[i][X.ix[i] == 1].index[0]
        disto = Distance.ix[0][gato]
        total_disto = disto*Passo
        CostOrigin = CostOrigin + total_disto
    
    # Calculate the intial cost of transfering passengers if any
    if pi > 0:
        for i in np.arange(Arrival_shape):
            for j in list(ind):
                if P.ix[i][j] > 0:
                    Passt = P.ix[i][j]
                    gati = X.ix[i][X.ix[i] == 1].index[0]
                    gatj = X.ix[j][X.ix[j] == 1].index[0]
                    distt = Distance.ix[gati][gatj]
                    total_distt = distt*Passt
                    CostTransfer = CostTransfer + total_distt
    
    # Total Cost 
    TotalCost =  CostOrigin + CostTransfer
    
    # Output
    return {'CostOrigin': CostOrigin, 'CostTransfer': CostTransfer, 'TotalCost': TotalCost, 'Gates': Gates}

In [6]:
# Below is a fucntion to Create a Neighbor

def CreateNeighbor(x):
    
    
    # SWAP
    def Swap(x):
        xx = copy.deepcopy(x['position'])
        #xx = q0
        n = len(xx) 
        i = random.sample(range(0,n), 2)
        i1 = i[0]
        i2 = i[1]
        xx[i2], xx[i1] = xx[i1], xx[i2]

        return  {'xx': xx}
   

    # REVERSION
    def Reversion(x):
        xx = copy.deepcopy(x['position'])
        #xx = q0
        n = len(xx) 
        i = random.sample(range(0,n), 2)
        i2 = max(i)
        i1 = min(i)
        rev = xx[i1:i2]
        rev = rev[::-1]
        xx[i1:i2] = rev


        return  {'xx': xx}

    
    # INSERSION
    def Insertion(x):
        xx = copy.deepcopy(x['position'])
        #xx = q0
        n = len(xx) 
        i = random.sample(range(0,n), 2)
        i2 = max(i)
        i1 = min(i)
        xx.insert(i2, xx[i1])
        del xx[i1]

        return  {'xx': xx}


    Choice = random.choice([1, 2, 3])
    if Choice == 1:
        return Swap(x)
    elif Choice == 2:
        return Reversion(x)
    else:
        return Insertion(x)

In [7]:
# Below is the function which checks the feasibility of the newly created neighbor

def CheckNewSol(Model, xt):
    
    q0 = xt['xx']
    Flights = Model['Flights']
    I = Model['I']
    J = Model['J']
    
    # Check the Feasiblity of the new solution created
    
    q0 = Series(q0)
    DelPos = q0.index[q0 > I - 1].tolist()
    DelPos

    Gate1 = q0[:DelPos[0]]
    Gate2 = q0[DelPos[0] + 1:DelPos[1]]
    Gate3 = q0[DelPos[1] + 1:DelPos[2]]
    Gate4 = q0[DelPos[2] + 1:DelPos[3]]
    Gate5 = q0[DelPos[3] + 1:DelPos[4]]
    Gate6 = q0[DelPos[4] + 1:DelPos[5]]
    Gate7 = q0[DelPos[5] + 1:DelPos[6]]
    Gate8 = q0[DelPos[6] + 1:DelPos[7]]
    Gate9 = q0[DelPos[7] + 1:DelPos[8]]
    Gate10 = q0[DelPos[8] + 1:DelPos[9]]
    Gate11 = q0[DelPos[9] + 1:DelPos[10]]
    Gate12 = q0[DelPos[10] + 1:DelPos[11]]
    Gate13 = q0[DelPos[11] + 1:DelPos[12]]
    Gate14 = q0[DelPos[12] + 1:]

    Gates = [list(Gate1), list(Gate2), list(Gate3), list(Gate4), list(Gate5), list(Gate6), list(Gate7), list(Gate8), 
             list(Gate9), list(Gate10), list(Gate11), list(Gate12), list(Gate13), list(Gate14)]
    
    gate_emp = []
    for i in Gates:
        L = len(i)
        gate_emp.append(L) 

    clash = 0
    for g in Gates:    
        for i in g:
            ai = Flights['Arrival Time'][i]
            di = Flights['Departure Time'][i]
            for j in g:
                if i == j:
                    continue
                else:
                    aj = Flights['Arrival Time'][j]
                    dj = Flights['Departure Time'][j]
                    if (dj - ai) * (di - aj) <= 0:
                        continue
                    else:
                        clash = clash + 1
    clash = clash/2
    if clash <= 0: 
        temp = "Pass"
    else:
        temp = "Fail"
        
    empty = min(gate_emp)
    
    # Output
    return {'Gates': Gates, 'clash': clash, 'temp': temp, 'empty': empty, 'q': q0}

In [8]:
# This is the main Cost Function

def MyCost(Model, NewFea):
    
    global NFE
    NFE = NFE + 1
    
    Flights = Model['Flights']
    Distance = Model['Distance']
    I = Model['I']
    J = Model['J']
    Arrival_shape = Model['Arrival_shape']
    P = Model['P']
    ind = Model['ind']
    Gates = NewFea['Gates']
    pi = Model['pi']
    
    # Create a new Xij based upon the new Neighbor
    X = DataFrame(0, index= np.arange(I), columns = np.arange(1,J + 1))
    for i in np.arange(1,J + 1):
        for j in (Gates[i - 1]):
            X[i][j] = 1
    
    
    CostOrigin = 0
    CostTransfer = 0
    
    # Calulate the Intial Cost of arriving and departing passengers
    for i in np.arange(Flights.shape[0]):
        xo = Flights.ix[i]
        Passo = xo['Passengers']
        gato = X.ix[i][X.ix[i] == 1].index[0]
        disto = Distance.ix[0][gato]
        total_disto = disto*Passo
        CostOrigin = CostOrigin + total_disto
    
    # Calculate the intial cost of transfering passengers if any
    if pi > 0:
        for i in np.arange(Arrival_shape):
            for j in list(ind):
                if P.ix[i][j] > 0:
                    Passt = P.ix[i][j]
                    gati = X.ix[i][X.ix[i] == 1].index[0]
                    gatj = X.ix[j][X.ix[j] == 1].index[0]
                    distt = Distance.ix[gati][gatj]
                    total_distt = distt*Passt
                    CostTransfer = CostTransfer + total_distt

    TotalCost = CostOrigin + CostTransfer
    
    return {'CostOrigin': CostOrigin, 'CostTransfer': CostTransfer, 'TotalCost': TotalCost, 'NFE': NFE, 'X': X, 'Gates': Gates}


In [9]:
# Below are some functions which do the plotting

# Plot Cost vs NFE
def PlotNFE(nfe, BestCost):
    sns.set_context("talk")
    fig1, ax = plt.subplots(1,1)
    plt.plot(nfe, BestCost, linestyle = '-', color = 'g')
    plt.xlabel('NFE')
    plt.ylabel('Cost')
    plt.title('Plot of Cost vs NFE')  

    
# Plot the sequence of fLights arranged at gates
def PlotEndResult(BestSol, Model):
    sns.set_context("poster")
    fig2, ax = plt.subplots(1,1)
    Xo = BestSol['X'].copy(deep=True) 
    for i in np.arange(Model['Flights'].shape[0]):
        arr = Model['Flights'].iloc[i]['Arrival Time']
        Xo.iloc[i][Xo.iloc[i] == 1] = arr
    Xo = Xo.replace(0,'NaN')
    ax = sns.stripplot(data=Xo, size=25, linewidth=1.2)
    plt.ylim(-45, 1600)
    plt.yticks(np.arange(0, 1600, 120))
    ax.set_yticklabels(['0:00','2:00','4:00','6:00','8:00','10:00','12:00','14:00','16:00','18:00',
                        '20:00','22:00','24:00','Next Day'])
    ax.set(xlabel='Gates', ylabel='Time', title = 'Flight Distribution on each Gate (End Result)')
    for a1 in np.arange(1,15):
        for b1 in np.arange(Xo.shape[0]):
            if Xo.iloc[b1][a1] > 0:
                if b1 < 10:
                    ax.annotate(b1, xy=(a1-1, Xo.iloc[b1][a1]), xytext=(-5,-4), textcoords='offset points', size=13)
                elif b1 < 100:
                    ax.annotate(b1, xy=(a1-1, Xo.iloc[b1][a1]), xytext=(-7.5,-4), textcoords='offset points', size=13)
                else:
                    ax.annotate(b1, xy=(a1-1, Xo.iloc[b1][a1]), xytext=(-11.25,-4), textcoords='offset points', size=13)

                    
# Plot the Distance of all gates from Origin
def PlotDistance(Model):
    sns.set_context("talk")
    fig3, ax = plt.subplots(1,1)
    ds = list(Model['Distance'][0]) 
    ds = ds[1:]
    Nd = len(ds)
    xd = range(1,Nd+1)
    axd = sns.barplot(xd,ds, color= 'green', saturation=.5)
    axd.set(xlabel='Gates', ylabel='Distance', title = 'Distance of Each Gate from Origin or Exit')

    
# Plot top 25 trasnfers if any
def PlotTrasnfer(Model):
    ps = Model['P'].sum()
    if ps.sum() == 0:
        print 
        print "No transfer passengers available"
    else:
        sns.set_context("talk")
        fig4, ax = plt.subplots(1,1)
        psort = ps.sort_values(ascending= False)[:25]
        indexx = psort.index.values.tolist()
        ls = list(psort)
        ax1 = sns.barplot(indexx, ls, color= 'red', saturation=.5, order = indexx)
        ax1.set(xlabel='Flight', ylabel='Passengers', title = 'Top 25 Flights having max transfer passenger')
        plt.xticks(rotation=45)
        
# Plot the Inital allotment of flights to gates
def PlotInitialSol(qi, Model):
    sns.set_context("poster")
    fig5, ax = plt.subplots(1,1)
    Qi = qi['X'].copy(deep=True) 

    for i in np.arange(Model['Flights'].shape[0]):
        arr2 = Model['Flights'].iloc[i]['Arrival Time']
        Qi.iloc[i][Qi.iloc[i] == 1] = arr2

    Qi = Qi.replace(0,'NaN')
    ax1 = sns.stripplot(data=Qi, size=25, linewidth=1.2)
    plt.ylim(-45, 1600)
    plt.yticks(np.arange(0, 1600, 120))
    ax1.set_yticklabels(['0:00','2:00','4:00','6:00','8:00','10:00','12:00','14:00','16:00','18:00',
                         '20:00','22:00','24:00','Next Day'])
    ax1.set(xlabel='Gates', ylabel='Time', title = 'Flight Distribution on each Gate (Initial Arrangement)')
    for a in np.arange(1,15):
        for b in np.arange(Qi.shape[0]):
            if Qi.iloc[b][a] > 0:
                if b < 10:
                    ax1.annotate(b, xy=(a-1, Qi.iloc[b][a]), xytext=(-5,-4), textcoords='offset points', size=13)
                elif b < 100:
                    ax1.annotate(b, xy=(a-1, Qi.iloc[b][a]), xytext=(-7.5,-4), textcoords='offset points', size=13)
                else:
                    ax1.annotate(b, xy=(a-1, Qi.iloc[b][a]), xytext=(-11.25,-4), textcoords='offset points', size=13)

In [ ]:
###### This is the main SA code####

NFE = 0


# Day and Transfer Passengers

Day = 1           # User can give a input from 1 to 7 for specific days
Transfer = 0.25      # User can give transfer passensger information between 0 to 0.99 



## SA Parameters ##

MaxIt = 3        # Maximum Number of Iterations
MaxIt2 = 10      # Maximum Number of Inner Iterations
T0 = 10          # Initial Temperature
alpha = 0.99     # Temperature Damping Rate



## Problem Defination ##

Model = CreateModel(Day, Transfer)
nVar = Model['nVar']
VarSize = [0]* nVar


## Initialization ##

# Create intial solution

qi = CreateInitialSol(Model)

InitialFea = CheckInitialSol(Model, qi)
if InitialFea['temp']  == 'Pass':
    print "The inital solution is feasible"
c = IntialCost(Model, qi)
x = {'position': qi['q'], 'cost': c['TotalCost'], 'Gates': c['Gates'], 'X': qi['X']} 


# Update Best Solution Ever Found
BestSol = x

# Array to Hold Best Cost Values
BestCost = [0]* MaxIt

# Array to Hold NFEs
nfe = [0]* MaxIt


# Set Initial Temperature
T = T0


# SA Main Loop

for it in np.arange(MaxIt):
    for it2 in np.arange(MaxIt2):
        
        # Keep creating a new neighbor unless a new feasible solution is generated  
        con = 'Fail' 
        while con == 'Fail':
            xt = CreateNeighbor(x)
            NewFea = CheckNewSol(Model, xt)
            con = NewFea['temp']
            
        # Calculate cost of the new feasible solution
        new_c = MyCost(Model, NewFea)
        x_new = {'position': xt['xx'], 'cost': new_c['TotalCost'], 'Gates':new_c['Gates'], 'X': new_c['X']} 

        if x_new['cost'] <= x['cost']:
            # xnew is better, so it is accepted
            x = x_new    
        else:
            # xnew is not better, so it is accepted conditionally
            delta = x_new['cost'] - x['cost']
            p = exp(-delta / T)
            
            rand = random.uniform(0, 1)
            if rand <= p:
                x = x_new
        
        # Update Best Solution  
        if x['cost'] <= BestSol['cost']:
            BestSol = x
            
   
    # Store Best Cost
    BestCost[it] = BestSol['cost']
    
    # Store NFE
    nfe[it] = NFE
    
    # Display Iteration Information
    text = 'Iteration ' + str(it) + ': NFE = ' + str(NFE) + ', Best Cost ' + str(BestCost[it])
    print text
    
    # Reduce Temperature
    T = T * alpha

print
print "Transfer passenegers were at most ", Transfer*100, "% per flight"  
print "Flight Sequence (Gate: Flight Number) --- Total Flights"
print
print "Gate1:", BestSol['Gates'][0], "-------",len(BestSol['Gates'][0])
print "Gate2:", BestSol['Gates'][1], "-------",len(BestSol['Gates'][1])
print "Gate3:", BestSol['Gates'][2], "-------",len(BestSol['Gates'][2])
print "Gate4:", BestSol['Gates'][3], "-------",len(BestSol['Gates'][3])
print "Gate5:", BestSol['Gates'][4], "-------",len(BestSol['Gates'][4])
print "Gate6:", BestSol['Gates'][5], "-------",len(BestSol['Gates'][5])
print "Gate7:", BestSol['Gates'][6], "-------",len(BestSol['Gates'][6])
print "Gate8:", BestSol['Gates'][7] , "-------",len(BestSol['Gates'][7])
print "Gate9:", BestSol['Gates'][8], "-------",len(BestSol['Gates'][8])
print "Gate10:", BestSol['Gates'][9], "-------",len(BestSol['Gates'][9])
print "Gate11:", BestSol['Gates'][10], "-------",len(BestSol['Gates'][10])
print "Gate12:", BestSol['Gates'][11], "-------",len(BestSol['Gates'][11])
print "Gate13:", BestSol['Gates'][12], "-------",len(BestSol['Gates'][12])
print "Gate14:", BestSol['Gates'][13], "-------",len(BestSol['Gates'][13])


# Create Plots  

PlotNFE(nfe, BestCost)
PlotInitialSol(qi, Model)
PlotEndResult(BestSol, Model)
PlotDistance(Model)
PlotTrasnfer(Model)